In [1]:
from selenium import webdriver
import pandas as pd
from scrap_article_new_tab import scrap_article
# from threading import Thread

from datetime import datetime as dt
from datetime import timedelta as td
from os import makedirs
from os.path import exists
from time import sleep
from json import loads

In [2]:
now = dt.now()
year = now.year
month = now.month
day = now.day
ts = str(int(round(now.timestamp())))

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
chrome = webdriver.Chrome(options=options)

In [4]:
# chrome = webdriver.Chrome()#chrome = webdriver.Chrome()

In [5]:
chrome.get(url = 'http://wyborcza.pl/0,75248.html')

In [6]:
entries = []
for entry in chrome\
    .find_element_by_xpath('//*[@id="pagetype_index"]/section[5]/div/main/div/div[2]/div/div[2]/ul')\
    .find_elements_by_class_name('entry'):
    entryContainer = {}
    
    entryContainer['author'] = entry.find_element_by_class_name('author').text.replace('|', '').strip()
    header = entry.find_element_by_tag_name('h3')\
                .find_element_by_tag_name('a')
    entryContainer['title'] = header.get_attribute('title')
    entryContainer['url'] = header.get_attribute('href')
    entryContainer['article_id'] = entryContainer['url'].split('/')[-1].split(',')[2]
    entryContainer['lead'] = entry.find_element_by_tag_name('p').text
    entryContainer['date'] = entry.find_element_by_class_name('when').text
    
    entries.append(entryContainer)

In [7]:
# try to click rodo (first click opens sometimes an ad)
for i in range(3):
    try:
        sleep(3)
        chrome.find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[1]/div/p[2]')\
            .click()
    except:
        pass

In [8]:
# close new window
try:
    chrome.switch_to.window(chrome.window_handles[1])
    chrome.close()
except:
    pass

In [9]:
scrapped_articles = pd.DataFrame(entries)
scrapped_articles['is_new'] = True
scrapped_articles['content'] = None

In [10]:
#scrapped_articles.head(5)

article_id               author              date  \
0   24814732   Jolanta Kowalewska  22-05-2019 18:39   
1   24814846        Piotr Szostak  22-05-2019 18:42   
2   24814740     Maciej Czarnecki  22-05-2019 18:26   
3   24814686  Wojciech Karpieszuk  22-05-2019 18:10   
4   24814733         Marek Deryło  22-05-2019 17:46   

                                                lead  \
0  Klub informuje, że to tylko część roszczeń. Za...   
1  "Nie uczyniliśmy wszystkiego, aby zapobiec krz...   
2  W ostatnich sondażach na Wyspach Brytyjskich p...   
3  Radni na najbliższej sesji nie zagłosują w spr...   
4  We wtorek w Warszawie odsłonięto pomnik Feliks...   

                                               title  \
0  Siatkarski klub Stocznia Szczecin pozwał Stocz...   
1  Episkopat: Nie ma słów, aby wyrazić nasz wstyd...   
2  Eurowybory wstrząsną sceną polityczną w Wielki...   
3  Głódź i Hoser na razie pozostaną honorowymi ob...   
4  Oglądałem, jak łomotaliście Ruskich. Czy Felik...   

                                                 url  is_new content  
0  http://szczecin.wyborcza.pl/szczecin/7,34939,2...    True    None  
1  http://wyborcza.pl/7,75398,24814846,episkopat-...    True    None  
2  http://wyborcza.pl/7,75399,24814740,eurowybory...    True    None  
3  http://warszawa.wyborcza.pl/warszawa/7,54420,2...    True    None  
4  http://wyborcza.pl/7,154903,24814733,ogladalem...    True    None

In [11]:
existing_articles = pd.read_csv('articles.csv')

In [12]:
#existing_articles.head(5)

article_id              author  \
0    24793013  Piotr Andrusieczko   
1    24792595      Magdalena Mach   
2    24791353    Jarosław Bińczyk   
3    24792543    Maciej Stasiński   
4    24793246                 key   

                                             content  \
0  Wyznaczenie terminu zaprzysiężenia prezydenta ...   
1  O pierwszym ataku policjanci zostali poinformo...   
2  Gdy reprezentacja zdobywała dwa razy mistrzost...   
3  Na rozmowy mediacyjne do Oslo zaprosił Wenezue...   
4  Na bydgoskiego biskupa Jana Tyrawę spadły grom...   

                         date                 division  \
0  2019-05-16 16:51:00.000000                    Świat   
1  2019-05-16 16:26:00.000000      podkarpacka policja   
2  2019-05-16 16:35:00.000000                siatkówka   
3  2019-05-16 15:47:00.000000                    Świat   
4  2019-05-16 15:34:00.000000  Wiadomości z Bydgoszczy   

                                           highlight  is_new  \
0  W czwartek Rada Najwyższa Ukrainy wyznaczyła i...   False   
1  Policjanci zatrzymali mężczyznę podejrzanego o...   False   
2  Z siatkarskiej PlusLigi znikają znani włoscy t...   False   
3  To kolejna próba namówienia obu stron - od trz...   False   
4  Coraz więcej trudnych pytań do bydgoskiego bis...   False   

                    last_date  \
0  2019-05-20 19:02:21.381542   
1  2019-05-20 19:02:21.381542   
2  2019-05-20 19:02:21.381542   
3  2019-05-20 19:02:21.381542   
4  2019-05-20 19:02:21.381542   

                                                lead  \
0  W czwartek Rada Najwyższa Ukrainy wyznaczyła i...   
1  Policjanci zatrzymali mężczyznę podejrzanego o...   
2  Z siatkarskiej PlusLigi znikają znani włoscy t...   
3  To kolejna próba namówienia obu stron - od trz...   
4  Coraz więcej trudnych pytań do bydgoskiego bis...   

                                          media_desc  \
0       Ukraiński prezydent elekt Wołodymyr Zełenski   
1  Zatrzymanie mężczyzny, podejrzanego o ataki na...   
2                                                NaN   
3  Przywódca wenezuelskiej opozycji, tymczasowy p...   
4                                  Biskup Jan Tyrawa   

                                           media_src       media_type  \
0  https://bi.im-g.pl/im/95/a4/17/z24792725V,Ukra...  multiple_images   
1  https://bi.im-g.pl/im/30/a4/17/z24792624V,Zatr...            image   
2      https://bi.im-g.pl/im/47/a4/17/z24791111V.jpg            image   
3  https://bi.im-g.pl/im/41/a4/17/z24792129V,Przy...            image   
4  https://bi.im-g.pl/im/78/1a/17/z24226168V,Bisk...            image   

                                               title  \
0  Data inauguracji Wołodymyra Zełenskiego w końc...   
1  Atakował kobiety w parku w centrum Rzeszowa. B...   
2  Schyłek ery w siatkówce? Polska jako etap prze...   
3  W Norwegii poufne rokowania z reżimem Nicolása...   
4  Bydgoski poseł pyta biskupa Tyrawę o jeszcze j...   

                                                 url  
0  http://wyborcza.pl/7,75399,24793013,data-inaug...  
1  http://rzeszow.wyborcza.pl/rzeszow/7,34962,247...  
2  http://wyborcza.pl/7,154903,24791353,schylek-e...  
3  http://wyborcza.pl/7,75399,24792543,w-norwegii...  
4  http://bydgoszcz.wyborcza.pl/bydgoszcz/7,48722...

In [13]:
len(scrapped_articles[~scrapped_articles['article_id'].isin(existing_articles['article_id'].unique())])

25

In [14]:
len(existing_articles)

5

In [15]:
combined_articles = existing_articles.append(scrapped_articles, sort=False)

In [16]:
len(combined_articles)

30

In [17]:
all_articles = combined_articles.drop_duplicates(keep='first')

In [18]:
all_articles['date'] = pd.to_datetime(all_articles['date'])

In [19]:
articles_to_drop = all_articles[
    all_articles['date'] <= dt.now() - td(days=7)
]

In [24]:
if articles_to_drop.empty == True:
    pass
else:
    existing_articles_to_drop = pd.read_csv('old_articles.csv')
    existing_articles_to_drop = existing_articles_to_drop.append(articles_to_drop)
    # backup file
    filename_to_save = 'data/backup/old/{}/{}/{}/{}.csv'.format(
        year, month, day, ts
    )
    if not exists(filename_to_save):
        makedirs(filename_to_save)
    existing_articles_to_drop.to_csv(filename_to_save, index=False)
    # save used in script file
    existing_articles_to_drop.to_csv('data/old_articles.csv', index=False)

In [21]:
articles_to_continue = all_articles[
    all_articles['date'] > dt.now() - td(days=7)
]

In [22]:
#  --------------------- TMP ---------------------
articles_to_continue['is_new'] = True
articles_to_continue['last_date'] = dt.now() - td(minutes=60)

In [23]:
#articles_to_continue.head(1)

article_id              author  \
0   24793013  Piotr Andrusieczko   

                                             content                date  \
0  Wyznaczenie terminu zaprzysiężenia prezydenta ... 2019-05-16 16:51:00   

  division                                          highlight  is_new  \
0    Świat  W czwartek Rada Najwyższa Ukrainy wyznaczyła i...    True   

                   last_date  \
0 2019-05-22 18:50:15.493092   

                                                lead  \
0  W czwartek Rada Najwyższa Ukrainy wyznaczyła i...   

                                     media_desc  \
0  Ukraiński prezydent elekt Wołodymyr Zełenski   

                                           media_src       media_type  \
0  https://bi.im-g.pl/im/95/a4/17/z24792725V,Ukra...  multiple_images   

                                               title  \
0  Data inauguracji Wołodymyra Zełenskiego w końc...   

                                                 url  
0  http://wyborcza.pl/7,75399,24793013,data-inaug...

In [30]:
def check_date(last_date, pub_date):
    hours_since_pub = (dt.now() - pub_date).total_seconds() / 60 / 60
    if hours_since_pub < 2:
        interval = 5 # in minutes
    elif hours_since_pub < 5:
        interval = 10
    elif hours_since_pub < 10:
        interval = 20
    else:
        interval = 30
    
    if dt.now() - td(minutes=interval) < last_date:
        return True
    else:
        return False

In [27]:
def check_date(pub_date):
    days_since_pub = (dt.now() - pub_date).total_seconds() / 60 / 60 / 24
    if days_since_pub > 5:
        return True
    else:
        return False

In [28]:
articles_list = articles_to_continue.to_dict('rows')

In [29]:
# TMP
#options = webdriver.ChromeOptions()
#options.add_argument("--disable-popup-blocking")
#chrome = webdriver.Chrome()



In [30]:
#chrome.execute_script("window.close();")

In [31]:
#chrome.close()

In [32]:
# chrome.execute_script("window.open('{}','_blank');".format(article['url']))
# for i in range(10):
#     if chrome.execute_script("return document.readyState") != 'complete':
#         print('not ready')

In [33]:
#chrome.switch_to.window(chrome.window_handles[0])

In [37]:
chrome.switch_to.window(chrome.window_handles[0])

In [42]:
container = []
first = True
for article in articles_list:
    article_id = article['url'].split('/')[-1].split(',')[2]
    chrome.execute_script("window.open('{}','_blank');".format(article['url']))
    chrome.close()
    chrome.switch_to.window(chrome.window_handles[0])
    sleep(7)
    
    
        
    if check_date(article['date']):
        if article['is_new']:
            a = loads(scrap_article(chrome=chrome, first_time=True))
            article['article_id'] = article_id
            article['content'] = a['content']
            article['division'] = a['division']
            article['highlight'] = a['highlight']
            article['media_type'] = a['media_type']
            article['media_src'] = a['media_src']
            article['media_desc'] = a['media_desc']
            article['is_new'] = False
            with open('data/comments/'+article_id, 'w') as f:
                f.write(str({'row': {
                   'comments': a['comments'],
                   'timestamp': dt.timestamp(dt.now())
                   }
                }))
    container.append(article)


KeyboardInterrupt: 

In [43]:
articles_updated = pd.DataFrame(container)

In [44]:
if 'Unnamed: 0' in articles_updated.columns:
    articles_updated = articles_updated.drop('Unnamed: 0', axis=1)

In [45]:
articles_updated.head()

article_id              author  \
0   24793013  Piotr Andrusieczko   
1   24792595      Magdalena Mach   
2   24791353    Jarosław Bińczyk   

                                             content                date  \
0  Wyznaczenie terminu zaprzysiężenia prezydenta ... 2019-05-16 16:51:00   
1  O pierwszym ataku policjanci zostali poinformo... 2019-05-16 16:26:00   
2  Gdy reprezentacja zdobywała dwa razy mistrzost... 2019-05-16 16:35:00   

              division                                          highlight  \
0                Świat  W czwartek Rada Najwyższa Ukrainy wyznaczyła i...   
1  podkarpacka policja  Policjanci zatrzymali mężczyznę podejrzanego o...   
2            siatkówka  Z siatkarskiej PlusLigi znikają znani włoscy t...   

   is_new                  last_date  \
0   False 2019-05-22 18:50:15.493092   
1   False 2019-05-22 18:50:15.493092   
2   False 2019-05-22 18:50:15.493092   

                                                lead  \
0  W czwartek Rada Najwyższa Ukrainy wyznaczyła i...   
1  Policjanci zatrzymali mężczyznę podejrzanego o...   
2  Z siatkarskiej PlusLigi znikają znani włoscy t...   

                                          media_desc  \
0       Ukraiński prezydent elekt Wołodymyr Zełenski   
1  Zatrzymanie mężczyzny, podejrzanego o ataki na...   
2                                                      

                                           media_src       media_type  \
0  https://bi.im-g.pl/im/95/a4/17/z24792725V,Ukra...  multiple_images   
1  https://bi.im-g.pl/im/30/a4/17/z24792624V,Zatr...            image   
2      https://bi.im-g.pl/im/47/a4/17/z24791111V.jpg            image   

                                               title  \
0  Data inauguracji Wołodymyra Zełenskiego w końc...   
1  Atakował kobiety w parku w centrum Rzeszowa. B...   
2  Schyłek ery w siatkówce? Polska jako etap prze...   

                                                 url  
0  http://wyborcza.pl/7,75399,24793013,data-inaug...  
1  http://rzeszow.wyborcza.pl/rzeszow/7,34962,247...  
2  http://wyborcza.pl/7,154903,24791353,schylek-e...

In [46]:
filename_to_save = 'data/backup/existing/{}/{}/{}'.format(
        year, month, day, ts
    )
if not exists(filename_to_save):
    makedirs(filename_to_save)

In [47]:
articles_updated.to_csv('data/backup/existing/{}/{}/{}/{}.csv'.format(
        year, month, day, ts
    )
)

In [48]:
articles_updated.to_csv('articles.csv', index = False)